In [9]:
# hide
# default_exp utils

# Sciflow utils

In [10]:
# export

import os
import subprocess
from pathlib import Path

import nbdev
from data_access.dremio_access import DremioAccess
from fastcore.script import call_parse
from nbdev.clean import nbdev_clean_nbs
from nbqa.find_root import find_project_root

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Paths

In [59]:
#export


def lib_path(*lib_relative_path):
    lib_root_path = find_project_root(srcs=(str(Path('.').resolve()), ))
    return Path(os.path.join(lib_root_path, *lib_relative_path))

In [69]:
assert(str(lib_path("nbs")).endswith('sciflow/nbs'))
assert(Path('test/test_clustering.ipynb').resolve() == lib_path("nbs", "test", "test_clustering.ipynb"))

# Sciflow Clean

In [70]:
# export


@call_parse
def sciflow_clean():
    nbdev.clean.cell_metadata_keep = nbdev.clean.cell_metadata_keep + ["tags"]
    nbdev_clean_nbs()

# Headless Dremio Access

In [71]:
# export


def shell_source(script):
    """Sometime you want to emulate the action of "source" in bash,
    settings some environment variables. Here is a way to do it.
    From: https://stackoverflow.com/questions/7040592/calling-the-source-command-from-subprocess-popen
    """

    pipe = subprocess.Popen(". %s; env" % script, stdout=subprocess.PIPE, shell=True)
    output = pipe.communicate()[0]
    env = dict((line.split("=", 1) for line in output.decode("utf-8").splitlines()))
    os.environ.update(env)

In [72]:
shell_source("/home/jovyan/.profile")

In [73]:
# export


def load_dremio_access():
    if os.path.exists("/home/jovyan/.auth/dremio"):
        with open("/home/jovyan/.auth/dremio", "r") as pat_file:
            pat = pat_file.read().strip()
            DremioAccess._get_dremio_password = lambda x: pat
    else:
        shell_source("/home/jovyan/.profile")
        if "DREMIO_PWD" in os.environ:
            DremioAccess._get_dremio_password = lambda x: os.environ["DREMIO_PWD"]
    return DremioAccess()

In [74]:
dremio_access = load_dremio_access()

## Add Dremio Access to startup script

In [75]:
%%writefile ~/.ipython/profile_default/startup/002.py

import os


if os.path.exists('/home/jovyan/.auth/dremio'):
    with open('/home/jovyan/.auth/dremio', 'r') as pat_file:
        pat = pat_file.read().strip()
        DremioAccess._get_dremio_password = lambda x: pat
    dremio_access = DremioAccess()

Overwriting /home/jovyan/.ipython/profile_default/startup/002.py
